# Testing with `doctest`

Why do I maintain the habit of adding a 1 level header? Isn't it unnecessary, since notebooks themselves have a title?

Anyway...

Idly browsing [The Python Tutorial](https://docs.python.org/3/tutorial/index.html), and particularly it's [10. Brief Tour of the Standard Library](https://docs.python.org/3/tutorial/stdlib.html), I came across `doctest`, introduced in [10.11 Quality Control](https://docs.python.org/3/tutorial/stdlib.html#quality-control), together with `unittest` module which I am familiar with.

Doctest is cool, the idea is to use documentation as test. Like so.

In [1]:
import doctest

In [4]:
print(doctest.__doc__)

Module doctest -- a framework for running examples in docstrings.

In simplest use, end each module M to be tested with:

def _test():
    import doctest
    doctest.testmod()

if __name__ == "__main__":
    _test()

Then running the module as a script will cause the examples in the
docstrings to get executed and verified:

python M.py

This won't display anything unless an example fails, in which case the
failing example(s) and the cause(s) of the failure(s) are printed to stdout
(why not stderr? because stderr is a lame hack <0.2 wink>), and the final
line of output is "Test failed.".

Run it with the -v switch instead:

python M.py -v

and a detailed report of all examples tried is printed to stdout, along
with assorted summaries at the end.

You can force verbose mode by passing "verbose=True" to testmod, or prohibit
it by passing "verbose=False".  In either of those cases, sys.argv is not
examined by testmod.

There are a variety of other ways to run doctests, including integratio

So for instance

In [10]:
def average(values):
    """Computes the arithmetic mean of a list of numbers.
    
    >>> print(average([20, 30, 70]))
    40.0
    """
    return sum(values) / len(values)

And then

In [11]:
doctest.testmod()

TestResults(failed=0, attempted=1)

I like this stuff, because it keeps documentation, testing and code close. The way to think about it is testing of examples.

# Verbosity

Verbosity can be added with `verbose=True` to `doctest.testmod()`.

In [7]:
doctest.testmod(verbose=True)

Trying:
    print(average([20, 30, 70]))
Expecting:
    40.0
ok
1 items had no tests:
    __main__
1 items passed all tests:
   1 tests in __main__.average
1 tests in 2 items.
1 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=1)

# `doctest` documentation

The [module documentation](https://docs.python.org/3/library/doctest.html#module-doctest) is worth reading of course. Test can also come from a text file, via `doctest.testfile()`.

In [184]:
class Animal:
    """An animal.
    
    >>> a = Animal(12, "moo", ["grass mostly"])
    >>> a.hp
    12
    """
    def __init__(self, hp, sound, diet):
        self.sound = sound
        self.diet = diet
        self.hp = hp

    def damage(self, d):
        self.hp -= d
        print(self.sound)

        
class Rodent(Animal):
    def __init__(self, hp, sound, diet):
        super().__init__(hp, sound, diet)
        self.hp = hp


class Snakie(Animal):
    def __init__(self, hp, sound, diet, venomous=False):
        super().__init__(hp, sound, diet)
        self.sound = sound
        self.venomous = venomous
        
    def bite(self, victim):
        """Attack a victim and inflict damage.
        
        >>> r = Rodent(4, 'squeeeek', ['cheese', 'root'])
        >>> s = Snakie(5, 'hiss', (Rodent))
        >>> s.bite(r)
        squeeeek
        
        >>> r2 = Rodent(4, 'squeak squeak', ['nuts'])
        >>> r2.hp
        4
        
        >>> a = Animal(12, "moo", ["grass mostly"])
        >>> s.bite(a)
        Not eating <class '__main__.Animal'>
        """
        assert isinstance(victim, Animal)
        try:
            assert isinstance(victim, self.diet)
            victim.damage(2)
            if self.venomous:
                victim.damage(1)
        except AssertionError:
            print("Not eating {}".format(type(victim)))

In [185]:
r = Rodent(4, 'squeek', ['cheese', 'root'])
s = Snakie(5, 'hiss', (Rodent))
s.bite(r)

squeek


In [186]:
s2 = Snakie(12, "hiss hiss", (Rodent))
s2.bite(s)

Not eating <class '__main__.Snakie'>


In [187]:
doctest.testmod(verbose=True)

Trying:
    a = Animal(12, "moo", ["grass mostly"])
Expecting nothing
ok
Trying:
    a.hp
Expecting:
    12
ok
Trying:
    r = Rodent(4, 'squeeeek', ['cheese', 'root'])
Expecting nothing
ok
Trying:
    s = Snakie(5, 'hiss', (Rodent))
Expecting nothing
ok
Trying:
    s.bite(r)
Expecting:
    squeeeek
ok
Trying:
    r2 = Rodent(4, 'squeak squeak', ['nuts'])
Expecting nothing
ok
Trying:
    r2.hp
Expecting:
    4
ok
Trying:
    a = Animal(12, "moo", ["grass mostly"])
Expecting nothing
ok
Trying:
    s.bite(a)
Expecting:
    Not eating <class '__main__.Animal'>
ok
Trying:
    print(average([20, 30, 70]))
Expecting:
    40.0
ok
9 items had no tests:
    __main__
    __main__.Animal.__init__
    __main__.Animal.damage
    __main__.Rodent
    __main__.Rodent.__init__
    __main__.Snakie
    __main__.Snakie.__init__
    __main__._43
    __main__._43.__init__
3 items passed all tests:
   2 tests in __main__.Animal
   7 tests in __main__.Snakie.bite
   1 tests in __main__.average
10 tests in 12 

TestResults(failed=0, attempted=10)